In [1]:
import numpy as np

In [7]:
datas=open('./dinos.txt','r').read() #读取数据
datas=datas.lower()
print(datas)


aachenosaurus
aardonyx
abdallahsaurus
abelisaurus
abrictosaurus
abrosaurus
abydosaurus
acanthopholis
achelousaurus
acheroraptor
achillesaurus
achillobator
acristavus
acrocanthosaurus
acrotholus
actiosaurus
adamantisaurus
adasaurus
adelolophus
adeopapposaurus
aegyptosaurus
aeolosaurus
aepisaurus
aepyornithomimus
aerosteon
aetonyxafromimus
afrovenator
agathaumas
aggiosaurus
agilisaurus
agnosphitys
agrosaurus
agujaceratops
agustinia
ahshislepelta
airakoraptor
ajancingenia
ajkaceratops
alamosaurus
alaskacephale
albalophosaurus
albertaceratops
albertadromeus
albertavenator
albertonykus
albertosaurus
albinykus
albisaurus
alcovasaurus
alectrosaurus
aletopelta
algoasaurus
alioramus
aliwalia
allosaurus
almas
alnashetri
alocodon
altirhinus
altispinax
alvarezsaurus
alwalkeria
alxasaurus
amargasaurus
amargastegos
amargatitanis
amazonsaurus
ammosaurus
ampelosaurus
amphicoelias
amphicoelicaudia
amphisaurus
amtocephale
amtosaurus
amurosaurus
amygdalodon
anabisetia
anasazisaurus
anatosaurus
anatotitan

In [4]:
#转换为无序且不重复的列表
chars=list(set(datas))
print('chars:',chars)
data_size,vocab_size=len(datas),len(chars)
print('data_size: %d,vocab_size: %d'%(data_size,vocab_size))
char_to_ix={ch:i for i,ch in enumerate(sorted(chars))}
ix_to_char={i:ch for i,ch in enumerate(sorted(chars))}

chars: ['b', 'z', 'o', '\n', 'y', 't', 'p', 's', 'm', 'j', 'a', 'q', 'h', 'n', 'g', 'k', 'r', 'w', 'u', 'l', 'd', 'e', 'i', 'v', 'x', 'c', 'f']
data_size: 19909,vocab_size: 27


可以看到该数据集有19909条样本，即19909个恐龙名字。但是包含的单词（字母）数为27（因为英文字母26个，再加一个换行符\n)

In [9]:
#重新读取下数据并存放到list里
with open('dinos.txt') as f:
    examples=f.readlines()
examples=[x.lower().strip() for x in examples]
print(examples)

['aachenosaurus', 'aardonyx', 'abdallahsaurus', 'abelisaurus', 'abrictosaurus', 'abrosaurus', 'abydosaurus', 'acanthopholis', 'achelousaurus', 'acheroraptor', 'achillesaurus', 'achillobator', 'acristavus', 'acrocanthosaurus', 'acrotholus', 'actiosaurus', 'adamantisaurus', 'adasaurus', 'adelolophus', 'adeopapposaurus', 'aegyptosaurus', 'aeolosaurus', 'aepisaurus', 'aepyornithomimus', 'aerosteon', 'aetonyxafromimus', 'afrovenator', 'agathaumas', 'aggiosaurus', 'agilisaurus', 'agnosphitys', 'agrosaurus', 'agujaceratops', 'agustinia', 'ahshislepelta', 'airakoraptor', 'ajancingenia', 'ajkaceratops', 'alamosaurus', 'alaskacephale', 'albalophosaurus', 'albertaceratops', 'albertadromeus', 'albertavenator', 'albertonykus', 'albertosaurus', 'albinykus', 'albisaurus', 'alcovasaurus', 'alectrosaurus', 'aletopelta', 'algoasaurus', 'alioramus', 'aliwalia', 'allosaurus', 'almas', 'alnashetri', 'alocodon', 'altirhinus', 'altispinax', 'alvarezsaurus', 'alwalkeria', 'alxasaurus', 'amargasaurus', 'amarga

In [11]:
class rnnModel():
    def __init__(self,training_datas,ix_to_char,char_to_ix,vocab_size,cell_nums=50,seed=1):
        self.training_datas=training_datas
        self.ix_to_char=ix_to_char  #索引映射字符字典
        self.char_to_ix=char_to_ix  #字符映射索引字典
        self.vocab_size=vocab_size  #文本中唯一字符的数量
        self.cell_nums=cell_nums  #rnn中间层单元数量
        self.seed=seed  #随机数种子



    def initial_params(self,cell_nums,n_x,n_y):
        np.random.seed(self.seed)
        self.Wax=np.random.randn(cell_nums,n_x)*0.01
        self.Waa=np.random.randn(cell_nums,cell_nums)*0.01
        self.Wya=np.random.randn(n_y,cell_nums)*0.01
        self.ba=np.zeros((cell_nums,1))
        self.by=np.zeros((n_y,1))
        print("initial params complete!")



    def softmax(self,z):
        e_z=np.exp(z-np.max(z))
        return e_z/np.sum(e_z,axis=0)



    def GradientDescentOptimizer(self,learning_rate,iterations=10000,sample_name_nums=7):
        self.initial_params(self.cell_nums,self.vocab_size,self.vocab_size)
        #初始化a[0]
        a_prev=np.zeros((self.cell_nums,1))
        #打乱数据
        np.random.shuffle(self.training_datas)


        for iter in range(iterations):
            index=iter%len(self.training_datas)
            X=[None]+[char_to_ix[ch] for ch in self.training_datas[index]]
            Y=X[1:]+[char_to_ix['\n']]
            loss,caches=self.forward(X,Y,a_prev)
            gradients,aprev=self.backward(X,Y,caches)
            cliped_gradients=self.clip_gradients(gradients,5)
            self.updateParams(cliped_gradients,learning_rate)
            if iter%2000==0:
                print("第" + str(iter + 1) + "次迭代，损失值为：" + str(loss))
                for name in range(sample_name_nums):
                    sample_indices=self.sample()
                    self.print_sample(sample_indices)





    def forward(self,X,Y,a0):
        #X是一个单词，要遍历其中的每一个字母
        #x,a,y_hat保存的分别是每一步one-hot后的输入字符，输出a，和预测值y_hat
        x_ont_hot,a,y_hat={},{},{}
        a[-1]=np.copy(a0)
        loss=0
        for t in range(len(X)):
            # 遍历X的每一个字符
            x_ont_hot[t]=np.zeros((self.vocab_size,1))

            if X[t]!=None:
                # one-hot编码
                x_ont_hot[t][X[t]]=1
            a[t]=np.tanh(self.Waa.dot(a[t-1])+self.Wax.dot(x_ont_hot[t])+self.ba)
            y_hat[t]=self.softmax(self.Wya.dot(a[t])+self.by)
            loss-=np.log(y_hat[t][Y[t],0])
        caches=(y_hat,a,x_ont_hot)
        return loss,caches



    def backward(self,X,Y,caches):
        #y_hat里保存的是每一步的预测结果
        (y_hat,a,x)=caches
        dWya=np.zeros_like(self.Wya)
        dby=np.zeros_like(self.by)
        dWaa=np.zeros_like(self.Waa)
        dWax=np.zeros_like(self.Wax)
        dba=np.zeros_like(self.ba)
        #最后一层没有提供a给下一层用
        da_next=np.zeros_like(a[0])
        for t in reversed(range(len(X))):
            dy=np.copy(y_hat[t])
            #这里是softmax求导，softmax求导为A-Y,Y是one-hot后的向量，只有一个位置上是1，其他位置都是0
            dy[Y[t]]-=1
            dWya+=np.dot(dy,a[t].T)
            dby+=dy
            da=np.dot(self.Wya.T,dy)+da_next
            dz=np.multiply(1-np.square(a[t]),da)
            dWaa+=np.dot(dz,a[t-1].T)
            dWax+=np.dot(dz,x[t].T)
            dba+=dz
            da_next=np.dot(self.Waa.T,dz)
        gradients={'dWya':dWya,'dby':dby,'dWaa':dWaa,'dWax':dWax,'dba':dba}
        return gradients,a[len(X)-1]



    def clip_gradients(self,gradients,maxValue):
        for k,v in gradients.items():
            np.clip(v,-maxValue,maxValue,out=v)
        return gradients


    def updateParams(self,gradients,lr):
        self.Wya+=-lr*gradients['dWya']
        self.Waa+=-lr*gradients['dWaa']
        self.Wax+=-lr*gradients['dWax']
        self.ba+=-lr*gradients['dba']
        self.by+=-lr*gradients['dby']


    def sample(self):
        #初始化x和a_prev
        x=np.zeros((self.vocab_size,1))
        a_prev=np.zeros((self.cell_nums,1))
        #包含要生成的字符的索引列表
        indices=[]
        #检测换行标志，初始化为-1
        idx=-1
        
        counter=0
        newline_character=char_to_ix['\n']
        
        while (idx!=newline_character and counter<50):
            #前向传播
            a=np.tanh(np.dot(self.Waa,a_prev)+np.dot(self.Wax,x)+self.ba)
            y=self.softmax(np.dot(self.Wya,a)+self.by)
            #设定随机数种子
            np.random.rand(counter)
            idx=np.random.choice(list(range(self.vocab_size)),p=y.ravel())
            indices.append(idx)
            #将输入字符重写采样为与采样索引对应的字符
            x=np.zeros((self.vocab_size,1))
            x[idx]=1

            #更新a_prev
            a_prev=a
            counter+=1

        if (counter==50):
            indices.append(char_to_ix['\n'])

        return indices



    def print_sample(self,sample_indices):
        #将ix转为字符
        txt=''.join(self.ix_to_char[ix] for ix in sample_indices)
        #将首字母大写
        txt=txt[0].upper()+txt[1:]
        #因为txt里面包含了\n，所以这里end=''，不然会多换一行
        print(txt,end='')

In [12]:
rnn=rnnModel(examples,ix_to_char,char_to_ix,27)
rnn.GradientDescentOptimizer(0.01)

initial params complete!
第1次迭代，损失值为：42.84647912665833
Oatgu
Uosqrnwsmxzs
Wcxycmaeevtrzaxyrqvujnydjjsozgnxoimpdtig
Wdkxshzozcwmaw
Ockjztmzfzzjgiz
Fapwrexxoymyer
Gspywoxr
第2001次迭代，损失值为：40.71467241192424
Nuchenaleatgudor
Docbohomggor
Luycdetmgenxgisaurus
Uranbkaur
Todenha
Orutisioras
Anputybution
第4001次迭代，损失值为：25.81417641256855
Monrocorus
Parratothus
Vhbrongusus
Dristafihavos
Cracrrosaurus
Errinocerus
Bugnonosaurus
第6001次迭代，损失值为：22.10971110023741
Ditanchosaurus
Singatoneus
Lanyusaurus
Uaryrosante
Jipoalosaurus
Ardyrus
Uygashurus
第8001次迭代，损失值为：25.7161092379475
Ganochasaurus
Iaapintasaurus
Fhuritosaurus
Ntosogoptosalophosauruch
Vinshalosaurus
Poshaidenus
Bithianptornoasaurus
